In [10]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
 
def rs(f1, f2, f, vx_, N):
    for j in range(N):
        if (vx_[j] > 0.):
            f[j, :, :] = vx_[j] * f1[j, :, :]
        else:
            f[j, :, :] = vx_[j] * f2[j, :, :]
    return f
    
def minmod(a, b):
    return np.where((a*b > 0), np.where(((abs(a) <= abs(b)) & (a*b > 0)), a, b), 0)    
    
def minmod2(a, b):
    if ((abs(a) <= abs(b)) & (a*b > 0)):
        return a
    elif (a*b > 0):
        return b
    else:
        return 0

def maxmod(a, b):
    return np.where((a*b > 0), np.where(((abs(a) >= abs(b)) & (a*b > 0)), a, b), 0)

def minmod_lim(u):
    return minmod(u[1:-1] - u[:-2], u[2:] - u[1:-1])

def superbee(u):
    return maxmod(minmod(u[2:] - u[1:-1], 2*(u[1:-1] - u[:-2])),
                             minmod(2*(u[2:] - u[1:-1]), u[1:-1] - u[:-2]))

def F_m(vx, vy, vz, T, n, Mol):
    Rg = 8.3144598 / Mol
    return n * ((1. / (2. * np.pi * Rg * T)) ** (3. / 2.)) * (np.exp(-((vx)**2 + (vy)**2 + (vz)**2) / (2. * Rg * T)))

def J(f, vmax, N):
    #fundamental constants
    Na = 6.02214129e+23
    kB = 1.381e-23 # J * K
    
    #gas parameters
    Mol = 40e-3 # kg
    Rg = 8.3144598  / Mol  # J / (mol * K) 
    m = Mol / Na # kg
    
    Pr = 2. / 3.
    C = 144.4
    T_0 = 273.11
    mu_0 = 2.125e-05

    #vmax = np.sqrt(2 * Rg * T) * 4
    #L = 21
    
    hv = 2. * vmax / N
    vx_ = np.linspace(-vmax+hv/2, vmax-hv/2, N)

    vx, vy, vz = np.meshgrid(vx_, vx_, vx_, indexing='ij')

    assert np.all(vx[:,0,0] == vx_)
    assert np.all(vy[0,:,0] == vx_)
    assert np.all(vz[0,0,:] == vx_)

    n = (hv ** 3) * np.sum(f)

    ux = (1. / n) * (hv ** 3) * np.sum(vx * f)
    uy = (1. / n) * (hv ** 3) * np.sum(vy * f)
    uz = (1. / n) * (hv ** 3) * np.sum(vz * f)
    
    T = (1. / 3. / n / Rg) * ((hv ** 3) * np.sum((vx*vx + vy*vy + vz*vz) * f) - n * (ux*ux + uy*uy + uz*uz))

    Vx = vx - ux
    Vy = vy - uy
    Vz = vz - uz

    rho = m * n

    p = rho * Rg * T

    cx = Vx / ((2. * Rg * T) ** (1. / 2.))
    cy = Vy / ((2. * Rg * T) ** (1. / 2.))
    cz = Vz / ((2. * Rg * T) ** (1. / 2.))

    Sx = (1. / n) * (hv ** 3) * np.sum(cx * (cx*cx + cy*cy + cz*cz) * f)
    Sy = (1. / n) * (hv ** 3) * np.sum(cy * (cx*cx + cy*cy + cz*cz) * f)
    Sz = (1. / n) * (hv ** 3) * np.sum(cz * (cx*cx + cy*cy + cz*cz) * f)

    mu = mu_0 * ((T_0 + C) / (T + C)) * ((T / T_0) ** (3. / 2.))


    f_plus = F_m(Vx, Vy, Vz, T, n, Mol) * (1. + (4. / 5.) * (1. - Pr) * (cx*Sx + cy*Sy + cz*Sz) * (cx*cx + cy*cy + cz*cz - (5. / 2.)))

    J = (p / mu) * (f_plus - f)
    
    return J, T

In [22]:
def solver(x_l, x_r, L, Tau, CFL, vmax, N, n_l, u_l, T_l, M):
    Mol = 40e-3
    g = 5. / 3.
    
    h = (x_r - x_l) / L 
    tau = h * CFL / vmax
    
    x = np.linspace(x_l+h/2, x_r-h/2, L)
    
    t = 0.
    
    hv = 2. * vmax / N
    vx_ = np.linspace(-vmax+hv/2, vmax-hv/2, N)
    vx, vy, vz = np.meshgrid(vx_, vx_, vx_, indexing='ij')
    
    n_r = (g + 1.) * M * M / ((g - 1.) * M * M + 2.) * n_l
    u_r = ((g - 1.) * M * M + 2.) / ((g + 1.) * M * M) * u_l
    T_r = (2. * g * M * M - (g - 1.)) * ((g - 1.) * M * M + 2.) / ((g + 1) ** 2 * M * M) * T_l
    
    
    F_l = F_m(vx-u_l, vy, vz, T_l, n_l, Mol)
    F_r = F_m(vx-u_r, vy, vz, T_r, n_r, Mol)
    
    
    # initial condition 
    f = np.zeros((L, N, N, N))
    for i in range(L/2):
        f[i, :, :, :] = F_l
    for i in range(L/2+1, L):
        f[i, :, :, :] = F_r
    
    slope = np.zeros((L, N, N, N))
    tmp = np.zeros((L, N, N, N))
    f_l = np.zeros((L+1, N, N, N))
    f_r = np.zeros((L+1, N, N, N))
    Flow = np.zeros((L+1, N, N, N))
    RHS = np.zeros((L, N, N, N))
    
    Temp = np.zeros(L)
    
    while(t < Tau):
        t += tau
        # boundary condition
        f_l[0, :, :, :] = F_l
        f_r[L, :, :, :] = F_r
        # reconstruction
        # compute slopes
        
        for i in range(1, L-1):
            slope[i, :, :, :] = h * (f[i+1, :, :, :] - 2 * f[i, :, :, :] + f[i-1, :, :, :])
            
        for i in range(L):
            f_r[i, :, :, :] = f[i, :, :, :] - (h / 2) * slope[i, :, :, :]
        
        for i in range(1, L+1):
            f_l[i, :, :, :] = f[i-1, :, :, :] + (h / 2) * slope[i-1, :, :, :]
        
        # riemann solver - compute fluxes
        for i in range(L+1):
            Flow[i, :, :, :] = rs(f_l[i, :, :, :], f_r[i, :, :, :], Flow[i, :, :, :], vx_, N)
            
        # compute RHS
        for i in range(L):
            RHS[i, :, :, :] = (- Flow[i+1, :, :, :] + Flow[i, :, :, :]) / h# + J(f[i, :, :, :], vmax, N)[0]

        # update values
        for i in range(L):
            tmp[i, :, :, :] = f[i, :, :, :] + tau * RHS[i, :, :, :]
                
        f = tmp
    
    for i in range(L):
        Temp[i] = J(f[i, :, :, :], vmax, N)[1]
    
    return f, Temp

In [ ]:
L = 50
N = 50
vmax = 2000.

n_l = 2e+25 
u_l = 100.
T_l = 300.

x_l = -10.
x_r = 10.

Tau = .01

Mol = 40e-3
g = 5. / 3.
CFL = 0.5
M = 1.5
    
h = (x_r - x_l) / L 
tau = h * CFL / vmax
    
x = np.linspace(x_l+h/2, x_r-h/2, L)
    
t = 0.
    
hv = 2. * vmax / N
vx_ = np.linspace(-vmax+hv/2, vmax-hv/2, N)
vx, vy, vz = np.meshgrid(vx_, vx_, vx_, indexing='ij')
    
n_r = (g + 1.) * M * M / ((g - 1.) * M * M + 2.) * n_l
u_r = ((g - 1.) * M * M + 2.) / ((g + 1.) * M * M) * u_l
T_r = (2. * g * M * M - (g - 1.)) * ((g - 1.) * M * M + 2.) / ((g + 1) ** 2 * M * M) * T_l
   
    
F_l = F_m(vx-u_l, vy, vz, T_l, n_l, Mol)
F_r = F_m(vx-u_r, vy, vz, T_r, n_r, Mol)

J_ = J(F_l, vmax, N)

Flow = np.zeros((N, N, N))

Flow = rs(F_l, F_r, Flow, vx_, N)


Temp = solver(x_l = x_l, x_r = x_r, L = L, Tau = Tau, CFL = CFL, vmax = vmax,
           N = N, n_l = n_l, u_l = u_l, T_l = T_l, M = M)[1]

/home/egor/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in double_scalars
/home/egor/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in double_scalars
/home/egor/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in double_scalars
/home/egor/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in double_scalars
/home/egor/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in double_scalars
/home/egor/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in double_scalars
/home/egor/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in double_scalars
/home/egor/anaconda2/lib/python2.7/site-packages/ip

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
ax.plot(Temp)
print Temp
#ax.plot(F_l[:, N/2, N/2])
#ax.plot(F_r[:, N/2, N/2])
#ax.plot(Flow[:, N/2, N/2])

#print vx_

#ax.plot(F_r[:, 10, 10])
#print n_l, n_r
#print u_l, u_r
print T_l, T_r